In [ ]:
import requests
import csv
import lxml.html as lh
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objs as go
import warnings
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


warnings.filterwarnings('ignore')
pd.set_option('display.max_rows',None)

fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 15
fig_size[1] = 12
plt.rcParams["figure.figsize"] = fig_size

from IPython.display import Image
from IPython.display import HTML
from collections import Counter

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
teams = {'G2 Esports':'black',
         'Fnatic':'orange',
         'Splyce':'red',
         'FC Schalke 04':'cyan',
         'Origen':'blue',
         'Team Vitality':'yellow',
         'Misfits Gaming':'red',
         'SK Gaming':'black',
         'Rogue':'blue',
         'Excel Esports':'navy'}
switcher = {
    "G2 Esports": "G2",
    "Team Vitality": "VIT",
    "Rogue": "RGE",
    "Fnatic": "FNC",
    "FC Schalke 04": "S04",
    "SK Gaming": "SK",
    "Excel Esports": "XL",
    "Misfits Gaming": "MSF",
    "Origen": "OG",
    "Splyce": "SPY",
}
def swap(x):
    temp = x['Team1']
    x['Team1'] = x['Team2']
    x['Team2'] = temp
    x['Result'] = (False if (x['Result']) else True)
    return(x)

def swaps(x):
    df.loc[x] = swap(df.loc[x])


In [ ]:
df = pd.read_csv("lec_matchdata.csv")
df2 = pd.read_csv("lec_playerdata.csv")
df3 = pd.read_csv("lec_championdata.csv")

# The League of Legends European Championship (LEC)

***
## Introduction

2019 was a landmark year for the League of Legends Europeans Championship; Franchising was introduced which brought a multitude of new and old teams into the league. G2 Esports took home Europe's first major trophy since before the arrival of Chinese/Korean teams to the scene. Three fully-European teams were sent to the World Championship and all three made it out of groups for the first time in the league's history. G2 Esports were well beaten at the final hurdle, but the year was nonetheless a raging success. In this notebook I aim to give a brief overview of the year to help others understand the data

***
## League Format

The year is split into two 'splits', Spring & Summer, with a major international tournament after both. Each split starts with the *Regular Season*, a double best-of-1 round robin, meaning each team plays all other teams twice. The top 6 teams in the regular season go through to the *Playoffs*, where teams match up in best-of-5 series in a hybrid single- & double-elimination format. 

The winner of the Spring split plays against the rest of the international league winners at the *Mid-Season Invitational* while the winner of Summer goes to the *World Championship*, the most prestigious tournament in the entire scene. Each team gains *circuit* points based on their playoff placings throughout the year and the team with the most points accompanies the Summer split winner to worlds. The 3 next best teams play in the *Gauntlet* for the final spot at Worlds

***
## Side Selection

Every League game has two sides, 'Red' and 'Blue' (left and right sides of the map). Blue side gets to pick the first champion but red gets to pick the last and both sides have different positional advantages due to the layout of the map. Each team chooses their side in half of their games, while the opponent chooses in the other half

In [ ]:
sums = df[['Sel','Choice']].groupby('Sel').sum()
totals = df['Sel'].value_counts().sort_index()
winrate_totals=[sums.iloc[i]/totals[i] for i in range(10)]
bardata = pd.concat(winrate_totals,axis = 1,keys = [s.name for s in winrate_totals])

px.bar(bardata, x=bardata.columns, y=[bardata.values[0][i] for i in range(10)])

fig = go.Figure(data=[
    go.Bar(name='Blue Side', 
           x=bardata.columns, 
           y=[bardata.values[0][i] for i in range(10)],
           marker_color = "#1d0efd",
           textposition="auto",
           text=["%s%%" % round(100*bardata.values[0][i],1) for i in range(10)]),

    
    go.Bar(name='Red Side', 
           x=bardata.columns, 
           y=[1-bardata.values[0][i] for i in range(10)],
           marker_color = "#fd0e35")
])
fig.update_yaxes(showticklabels=False)
fig.update_layout(
    barmode='stack',
    title = "Side Selection for Each Team: Regular Season",
    font=dict(
        family="Calibri, monospace",
        size=16,
        color="#7f7f7f")
                 )
fig.show()

As we can see, most teams strongly prefer to pick blue side. Though Team Vitality do go against that trend.

***
## Casters

One of the most important aspects of the broadcast is the casters. Most games will pair a 'Play-by-play' commentator with a 'Colour' commentator (Commentary v Analysis). Every now and again, however, we will get a tri-cast with two colour casters to provide more analysis during the game. The treemap below shows the different combinations we had during the regular season. (Casters go by pseudonyms, similar to the players)

In [ ]:
def scores(team):
    for i in (df[df['Team2'] == team].index):
        swaps(i)
    team_results = df[df['Team1'] == team].append(df[df['Team2'] == team]).sort_values('UTC')
    score = team_results.groupby('Team1').cumsum().set_index(team_results['UTC'])
    return(score)

In [ ]:
casters=df[['Team1','Team2','PBP','Color']]
casters['Combo'] = casters['PBP'] + " + " + casters['Color']
caster_combos = casters.groupby('Combo').count()
test = casters.sort_values('Combo')['Combo'].drop_duplicates()
parents = ["",""]
for i in test:
    if("," in i):
        parents.append("Tri Cast")
    else:
        parents.append("Duo Cast")
labels_src = casters.sort_values("Combo")['Combo'].drop_duplicates()
labels = ["Tri Cast","Duo Cast"]
for label in labels_src:
    labels.append(label)

values = [0,0]
for value in caster_combos['Team1']:
    values.append(value)
    
fig = go.Figure(go.Treemap(nams=labels,
                           parents = parents,
                           values = values))
fig.update_layout(title="Caster Combo Distribution: Regular Season")
fig.show()

Interestingly, there doesn't seem to be a fully even distribution in terms of how many times each caster casts a certain team. Clicking on each caster's name here will show how many games they casted per team. For example, Quickshot only casted games involving Origen 4 times. Note that hovering over their names will show the total amount of regular season games they casted multiplied by 2, so halve the given number to get their personal total of games

In [ ]:
def casters_treemap(caster_type):
    pbp = pd.DataFrame([i for i in casters[caster_type].drop_duplicates() if ',' not in i],columns=[caster_type])
    pbp = pbp.rename(index=pbp[caster_type])
    pbp = pbp.drop(caster_type,axis=1)


    for team in teams:
        pbp[team] = 0
    for i in range(len(casters)):
        ind = casters.iloc[i]
        pbp_index = ind[caster_type].replace(" ","").split(",")
        for caster in pbp_index:    
            pbp.loc[caster,ind['Team1']] += 1
            pbp.loc[caster,ind['Team2']] += 1

    biglist = [len(pbp)*['o'],[10*[i] for i in list(pbp.index)]]
    parents = [val for sublist in biglist for val in sublist]
    parents = [val for sublist in parents for val in sublist]
    for i in range(len(pbp)):
        parents[i] = ""

    values = pbp.sum(axis=1)
    for i in list(pbp.index):
        values = values.append(pbp.loc[i])

    fig = go.Figure()

    fig = fig.add_trace(go.Treemap(
        labels = pbp.index.append(len(pbp)*[pbp.columns]),
        parents = parents,
        values = values,
        branchvalues= 'total'
    ))
    fig.update_layout(title='%s Team Distribution (Regular Season)' % caster_type)
    fig.show()
    
for i in ['PBP','Color']:
    casters_treemap(i)

In [ ]:
df = df.drop([90,181]) #dropping tiebreakers

***
## MVPs

Each regular season game gets an MVP (Most Valuable Player) who is voted in by the fans.

In [ ]:
mvps = df.groupby('MVP').count().sort_values('Team1',ascending = False)

fig = go.Figure()
fig.add_trace(go.Bar(y=mvps.index[range(5)][::-1], 
                     x=mvps['Team1'].head()[::-1],
                     orientation='h',
                    marker_color = "purple"))
fig.update_layout(title_text = 'Total MVP awards')
fig.show()

***
## Regular Season Wins

Next I wanted to see how consistent the teams were in the round robins. 

In [ ]:
df = df.drop("Unnamed: 0",axis = 1)
test = pd.DataFrame(columns = teams,index= df['UTC'])
for x in teams:
    test[x] = scores(x)['Result']
test = test.sort_values('UTC').drop_duplicates()

In [ ]:
fig = go.Figure(layout=go.Layout(
        title=go.layout.Title(text="Cumulative Wins: Regular Season Games")
    ))
def plotcumwins(x):
    fig.add_trace(go.Scatter(x=list(range(1,len(test[x])+1)),
                               y=test[x],
                               name = x,
                               line = dict(color=teams[x])))
for i in teams:
    plotcumwins(i)
fig.add_shape(go.layout.Shape(type="line",x0=18,y0=0,x1=18,y1=30,line=dict(
                color="RoyalBlue",
                width=1
            )))
fig.update_xaxes(title_text='Gameday',range=[0,36])
fig.update_yaxes(title_text='Total Number of Wins')
fig

Some interesting things to note from this include G2's sheer dominance (Which was mirrored in their playoff dominance) and the tussle that occured between Fnatic, Splyce, Schalke and OG throughout the year for the 2nd spot. Rogue's absymal spring season meant that even a Summer split resurgence and an eventual 4th place finish couldn't save them from being 10th place overall

***
## Player Nationalities

One thing that LEC fans are generally very proud of is the local talent that Europe produces. All 15 players sent to the World Championship this year were European, while 14/15 of the players sent last year were EU players. This season boasted a wide range of nationalities, with 22 European countries represented along with Korea.

In [ ]:
tops = df2[df2['Pos'] == "TOP"]
jgls = df2[df2['Pos'] == "Jungle"]
mids = df2[df2['Pos'] == "Middle"]
adcs = df2[df2['Pos'] == "ADC"]
sups = df2[df2['Pos'] == "Support"]

In [ ]:

img=mpimg.imread('/kaggle/input/lecnationalities/nationalities.png')
imgplot = plt.imshow(img)


As we can see, the Nordic and East region is responsible for a large amount of the players. That said, most of the Western countries and a large amount of Eastern countries have representation. It's worth noting that Korea clocks in at 10 players, higher than any single European country.

***
## Player Experience

Another thing that the league prides itself on is the integration of rookie players, players who haven't played professionally before. The European League scene has a thriving set of second division leagues and the majority of orgs are comfortable giving chances to unproven talent

In [ ]:
years = [0,2,1,2,1,4,1,2,0,1,2,3,4,0,6,2,2,0,4,4,1,5,0,1,5,8,3,4,3,3,3,6,3,0,0,1,6,4,0,1,3,3,2,4,7,0,0,3,8,0,1,5,4,2,4,3,2,0,0,2,4,0,0,3,0,0,0,4,1,0,4,5,0]
x = Counter(years)

fig = go.Figure(go.Pie(labels=["%s yr" % i for i in sorted(x.keys())],values=[x[i] for i in sorted(x.keys())],sort=False))
fig.update_layout(title_text="Years since Major Region Debut")
fig.show()

As we can see, fewer than half of the league's players in 2019 had their debut before 2017. Additionally, over a quarter of all players had no experience in major regions before featuring this year. With 10 new rookies already announced for the 2020 season, the future is looking up for the LEC.

In [ ]:
x = df2[['Player','Split']].groupby('Split').count().iloc[[2,4]]
fig = go.Figure(go.Bar(x = x.index, 
                       y = list(x['Player']),
                      marker_color = "#6baed6"))

fig.update_layout(title_text = "Number of Players per Split")
fig.update_yaxes(title_text="Number of Players")

fig.show()

***
## AD Carries

ADCs tend to be fed the most amount of gold throughout the game and are responsible for a large chunk of the teamfight damage. I wanted to explore how efficiently ADCs used the gold share they were given. The graph below shows how much of their team's damage they did compared to how much of the team's gold they got. Ideally you want your adc to deal as much damage as possible.

In [ ]:
fig = px.scatter(x=adcs['Gold%'],
                 y=adcs['DMG%'],
                 color=adcs['Split'],
                 hover_name=adcs['Player'])
fig.update_xaxes(title_text="Gold%",range=[35,15])
fig.update_yaxes(title_text="Damage%")
fig.update_layout(title_text="ADC Gold Efficiency")
fig.show()

***
## Midlaners

Midlaners are arguably the most important role in the modern game. They control the central lane of the map, meaning the game tends to flow through them. Having pressure in the midlane allows your team more freedom of movement while midlaners roaming can dictate a large part of the early game. I wanted to see how dominant certain midlaners are in the laning phase by looking at their average XP (experience) and CS (creep score, neutral minions killed) differential over their counterpart. A player who plays a strong lane or low roaming style will generally have high differentials

In [ ]:
fig = px.scatter(y=mids['GD10'],
                 x = mids['XPD10'],
                hover_name = mids['Player'],
                color = mids['Split'])

fig.update_xaxes(title_text="XP Difference at 10")
fig.update_yaxes(title_text="Gold Difference at 10")
fig.update_layout(title_text=" Midlaner Laning Dominance")
fig.show()

***
## Supports

Supports are the utility members of the team. They take no gold and are usually responsible for engaging/vision control. Vision has been a key part of the meta since the game began and supports have always been at the heart of it. Vision gives information, information wins games. Here we look at how much vision control supports have by looking at their average number of wards placed every minute versus the average number of enemy wards destroyed every minute. 

In [ ]:
fig = px.scatter(x=sups['WPM'],
                 y=sups['WCPM'],
                 color=sups['Split'],
                 hover_name=sups['Player'])
fig.update_xaxes(title_text="Wards Placed Per Minute")
fig.update_yaxes(title_text="Wards Cleared Per Minute")
fig.update_layout(title_text="Support Vision Control")
fig.show()

In [ ]:
time_splits = df3.columns[[15,18,21,24,27,30]]
times = pd.DataFrame(index = time_splits, columns = ["Spring","Summer"])
grouped_df3 = df3.groupby('Split').sum()
for i in time_splits:
    times['Spring'][i] = (grouped_df3[i][0] / 10) / 107
    times['Summer'][i] = (grouped_df3[i][1] / 10) / 114

times.columns
time_splits = [i.replace("games","minutes") for i in time_splits] 

***
## Game Time

This year has been described as having one of the fastest metas in history. Teams were constantly winning games before 30 minutes, a far cry from the snoozefest of 5 years ago

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='Spring', x=time_splits, y=times['Spring']),
    go.Bar(name='Summer', x=time_splits, y=times['Summer'])
])
fig.update_layout(title_text = "Game Length: Spring v Summer Split")
fig.update_yaxes(title_text = "Percentage of Games")
fig.show()

As we can see, the majority of games were finished before the 35 minute mark, while there were fewer games longer than 40 minutes in the Summer split. It seems that teams improved heavily on quickly closing out games in the second half of the year

In [ ]:
champs = df3[['Champion 1','∑ 2','W 3','L 4']].groupby('Champion 1').sum().sort_values("∑ 2",ascending = False)
x = df3[['W 3','Split']].groupby('Split').count()

fig = go.Figure(go.Bar(x = x.index, 
                       y = list(x['W 3']),
                      marker_color = "#f768a1"))

fig.update_layout(title_text = "Number of Unique Champions per Split")
fig.update_yaxes(title_text="Number of Champions")

fig.show()

***
## Champions

Champions are one of the most important part of the game, each player gets to choose one character with unique abilities to play the game as. Here we explore some of the basic graphs about these champions

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='Wins', 
           x=champs.index[range(10)], 
           y=champs['W 3'],
           marker_color = "#1c9099",
           textposition="auto"),

    
    go.Bar(name='Losses', 
           x=champs.index[range(10)], 
           y=champs['L 4'],
           marker_color = "#a6bddb",
           textposition = "auto",
            text = champs['∑ 2']),
          
])
fig.update_yaxes(title_text = "Number of Games")
fig.update_layout(
    barmode='stack',
    title = "Most Picked Champions: Spring & Summer Split",
    font=dict(
        family="Calibri, monospace",
        size=16,
        color="#7f7f7f")
                 )
fig.show()

In [ ]:
champs['WR'] = champs['W 3'] / champs['∑ 2']
champs = champs[champs['∑ 2'] > 10].sort_values('WR',ascending = False)

fig = go.Figure(data=[
    go.Bar(name='Winrate', 
           x=champs.index[range(10)], 
           y=champs['WR'],
           marker_color = "#fec44f",
           textposition="auto",
          text=["%s%%" % round(100*champs['WR'].iloc[i],1) for i in range(10)]),
])
fig.update_yaxes(title_text = "Win Percentage")
fig.update_layout(
    barmode='stack',
    title = "Highest Winrate Champions: Spring & Summer Split (Min 10 Games)",
    font=dict(
        family="Calibri, monospace",
        size=16,
        color="#7f7f7f")
                 )
fig.show()

In [ ]:
champs = champs[champs['∑ 2'] > 10].sort_values('WR')
fig = go.Figure(data=[
    go.Bar(name='Winrate', 
           x=champs.index[range(10)], 
           y=champs['WR'],
           marker_color = "#78c679",
           textposition="auto",
          text=["%s%%" % round(100*champs['WR'].iloc[i],1) for i in range(10)]),
])
fig.update_yaxes(title_text = "Win Percentage")
fig.update_layout(
    barmode='stack',
    title = "Lowest Winrate Champions: Spring & Summer Split (Min 10 Games)",
    font=dict(
        family="Calibri, monospace",
        size=16,
        color="#7f7f7f")
                 )
fig.show()

Finally, the most important stat. Often-ridiculed Support player 'PromisQ' picked up his first ever tournament win as G2's substitute support. He now owns more LEC medals than Fnatic, the most famous and successful team in Europe

In [ ]:
fig = go.Figure(go.Bar(x=["Fnatic","Promisq"],y=[0,2],marker_color="grey"))
fig.update_layout(title_text="Number of LEC Medals")
fig.show()